In [12]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd

class VehicleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
        
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')  # 读取图像并转换为RGB格式
        if self.transform:
            image = self.transform(image)  # 如果提供了transform就进行预处理
        label = self.labels[idx]  # 获取对应的标签
        return image, label  # 返回图像张量和标签

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, val_loader, device, verbose=False):
    model.eval()
    all_preds = []
    all_labels = []
    batch_accuracies = []
    batch_f1_scores = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # 计算每批次的accuracy和macro-F1
            batch_accuracy = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
            batch_f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            batch_accuracies.append(batch_accuracy)
            batch_f1_scores.append(batch_f1)

    # 计算整体accuracy和macro-F1
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    # 计算每批次的方差
    accuracy_variance = np.var(batch_accuracies)
    f1_variance = np.var(batch_f1_scores)

    if verbose:
        print(f"Batch Accuracies: {batch_accuracies}")
        print(f"Batch Macro-F1 Scores: {batch_f1_scores}")

    return accuracy, macro_f1, accuracy_variance, f1_variance

In [13]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # 加载数据
    data_dir = "Data/Data"  # 修正数据路径
    image_paths = []
    labels = []
    
    # 遍历文件夹收集图片路径和标签
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # 从文件名中提取类别标签（1_xxx.png -> 0, 2_xxx.png -> 1, 3_xxx.png -> 2）
            class_id = int(filename.split('_')[0]) - 1  # 将1,2,3映射到0,1,2
            image_paths.append(os.path.join(data_dir, filename))
            labels.append(class_id)
    
    # 转换为numpy数组
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    print(f"总样本数: {len(labels)}")
    print("类别分布:")
    for i in range(3):
        print(f"类别 {i+1}: {sum(labels == i)} 个样本")
    
    # 5折交叉验证
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
        print(f"\nFold {fold + 1}")
        
        # 准备数据集
        train_dataset = VehicleDataset(
            [image_paths[i] for i in train_idx],
            [labels[i] for i in train_idx],
            transform=transform
        )
        val_dataset = VehicleDataset(
            [image_paths[i] for i in val_idx],
            [labels[i] for i in val_idx],
            transform=transform
        )
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # 模型初始化
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 3)  # 3个类别
        model = model.to(device)
        
        # 损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_score = 0
        best_epoch = 0
        patience = 3
        no_improve = 0
        
     # 训练循环
    for epoch in range(25):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        accuracy, macro_f1, accuracy_variance, f1_variance = evaluate(model, val_loader, device)

        current_score = 0.7 * accuracy + 0.3 * macro_f1

        print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Score: {current_score:.4f}")

        # 早停策略
        if current_score > best_score:
            best_score = current_score
            best_epoch = epoch
            no_improve = 0

             # ✅ 保存当前最好的模型
            torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # ✅ 重新加载当前 fold 的最佳模型再评估
    model.load_state_dict(torch.load(f'best_model_fold{fold}.pth'))

    # 记录最终结果
    final_accuracy, final_macro_f1, final_accuracy_variance, final_f1_variance = evaluate(model, val_loader, device)
    accuracies.append(final_accuracy)
    f1_scores.append(final_macro_f1)

    print(f"\nFold {fold + 1} Final Results:")
    print(f"Accuracy: {final_accuracy:.4f}")
    print(f"Macro-F1: {final_macro_f1:.4f}")
    fold_score = 0.7 * final_accuracy + 0.3 * final_macro_f1
    print(f"Fold Score: {fold_score:.4f}")
    # 计算5折交叉验证的平均结果
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    print("\n最终五折交叉验证结果与指标:")
    print(f"Accuracy 均值: {mean_accuracy:.4f}, 方差: {std_accuracy:.4f}")
    print(f"Macro-F1 均值: {mean_f1:.4f}, 方差: {std_f1:.4f}")

    
    # 计算最终平均得分
    final_score = 0.7 * mean_accuracy + 0.3 * mean_f1
    print(f"\n最终平均得分: {final_score:.4f}")

if __name__ == "__main__":
    main()

使用设备: cuda
总样本数: 1572
类别分布:
类别 1: 720 个样本
类别 2: 608 个样本
类别 3: 244 个样本

Fold 1

Fold 2


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Fold 3

Fold 4

Fold 5
Epoch 1, Loss: 0.2769, Accuracy: 0.9140, Macro-F1: 0.9000, Score: 0.9098
Epoch 2, Loss: 0.1397, Accuracy: 0.9713, Macro-F1: 0.9667, Score: 0.9699
Epoch 3, Loss: 0.1333, Accuracy: 0.9682, Macro-F1: 0.9662, Score: 0.9676
Epoch 4, Loss: 0.0955, Accuracy: 0.9586, Macro-F1: 0.9586, Score: 0.9586
Epoch 5, Loss: 0.0496, Accuracy: 0.9554, Macro-F1: 0.9438, Score: 0.9519
Early stopping at epoch 5

Fold 5 Final Results:
Accuracy: 0.9618
Macro-F1: 0.9568
Fold Score: 0.9603

最终五折交叉验证结果与指标:
Accuracy 均值: 0.9618, 方差: 0.0000
Macro-F1 均值: 0.9568, 方差: 0.0000

最终平均得分: 0.9603
